Imports libraries

In [1]:
import pandas as pd
import joblib
from sklearn.metrics import roc_auc_score
import tensorflow as tf

In [2]:
# Load preprocessed data
%run ./03_preprocessing_and_dataset_summary.ipynb

# Load models
dt = joblib.load("results/decision_tree_model.pkl")
nn = tf.keras.models.load_model("results/neural_network_model.keras")


Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-n

Hyperparameter Settings

In [3]:
hyperparams = pd.DataFrame({
    'Model': ['Decision Tree', 'Decision Tree', 'Neural Network', 'Neural Network', 'Neural Network', 'Neural Network', 'Neural Network', 'Neural Network'],
    'Hyperparameter': ['max_depth', 'random_state', 'Layers', 'Activation Functions', 'Optimizer', 'Loss Function', 'Epochs', 'Validation Split'],
    'Value': [5, 42, '[32,16,1]', 'ReLU (hidden), Sigmoid (output)', 'Adam', 'Binary Crossentropy', 50, 0.2]
})
print(hyperparams)
hyperparams.to_csv("results/hyperparameter_settings.csv", index=False)

            Model        Hyperparameter                            Value
0   Decision Tree             max_depth                                5
1   Decision Tree          random_state                               42
2  Neural Network                Layers                        [32,16,1]
3  Neural Network  Activation Functions  ReLU (hidden), Sigmoid (output)
4  Neural Network             Optimizer                             Adam
5  Neural Network         Loss Function              Binary Crossentropy
6  Neural Network                Epochs                               50
7  Neural Network      Validation Split                              0.2


Model Comparison (ROC-AUC)

In [4]:
roc_auc_dt = roc_auc_score(y_test, dt.predict_proba(X_test)[:,1])
roc_auc_nn = roc_auc_score(y_test, nn.predict(X_test))
print(f"Decision Tree ROC-AUC: {roc_auc_dt:.3f}")
print(f"Neural Network ROC-AUC: {roc_auc_nn:.3f}")

45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Decision Tree ROC-AUC: 0.851
Neural Network ROC-AUC: 0.831


Predict a new customer

In [6]:
nn_model = None

def predict_new_customer(customer_dict, model_type="nn"):
    global nn_model
    df = pd.DataFrame([customer_dict])
    df = pd.get_dummies(df)
    df = df.reindex(columns=X_train.columns, fill_value=0)
    df[num_cols] = scaler.transform(df[num_cols])

    if model_type == "nn":
        if nn_model is None:
            nn_model = tf.keras.models.load_model("results/neural_network_model.keras")
        pred = (nn_model.predict(df) > 0.5).astype("int32")[0][0]
    else:
        pred = dt.predict(df)[0]

    return "Churn:Yes" if pred == 1 else "Churn:No"

# Example
customer_example = {
    'gender':'Female', 'SeniorCitizen':0, 'Partner':'Yes', 'Dependents':'No',
    'tenure':12, 'PhoneService':'Yes', 'MultipleLines':'No',
    'InternetService':'Fiber optic', 'OnlineSecurity':'No',
    'OnlineBackup':'Yes', 'DeviceProtection':'No',
    'TechSupport':'No', 'StreamingTV':'No', 'StreamingMovies':'No',
    'Contract':'Month-to-month', 'PaperlessBilling':'Yes', 'PaymentMethod':'Electronic check',
    'MonthlyCharges':70.35, 'TotalCharges':845.5
}

print("Prediction for new customer:", predict_new_customer(customer_example, model_type="nn"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
Prediction for new customer: Churn:No
